## Import Libraries

In [1]:
import os #to import all files in directory
import docx #to read docx files
import pandas as pd #to work with DataFrames

import time #to have today's date
timestr = time.strftime("%Y%m%d")

### import from other notebook

In [2]:
#get the clean text from the other notebook
%run ./p_01_import_transcripts_paper.ipynb
#the dictionares contain all the interviews and their names (keys), e.g. 'UK_05_poli'
# dict_all_interviews_a: only answers
# dict_all_interviews_qa: questions and answers combined
# dict_all_interviews_q_a : questions and answers seperately

## do overview of interviews

### overview of all Interviews

In [3]:
#prepare overview of interviews in one df
df_interviews_overview = pd.DataFrame()
for x in dict_all_interviews_qa:
    var_no_words = 0 #count the number of words in each interview
    for y in dict_all_interviews_qa[x].paragraphs:
        while '  ' in y.text:
            y.text = y.text.replace('  ', '') #remove double space, otherwise this might be counted as a word
            if '  ' not in y.text:
                break
        var_no_words += len(y.text.split(' '))

    var_no_paras = len(dict_all_interviews_qa[x].paragraphs)
    df_interviews_overview = df_interviews_overview.append({'Country':x.split('_')[0], 'No_per_Country':x.split('_')[1], 
            'Class':x.split('_')[2], 'Paragraphs':int(var_no_paras), 'Words':int(var_no_words),
            'av_words_per_para': round(var_no_words/var_no_paras,1)}, ignore_index=True)

In [4]:
df_interviews_overview.to_csv('.\\results\overview_interviews\qa_overview_interviews_all_'+timestr+'.csv', sep=';', decimal=',')

## grouped by country

In [5]:
#calculate per country mean paragraph number, min&max, words per paragraph av etc.
df_country_overview = pd.DataFrame()
for x in set(df_interviews_overview['Country']):
    df_count = df_interviews_overview[df_interviews_overview['Country']==x]
    var_no_int = len(df_count)
    var_tot_paras = df_count['Paragraphs'].sum()
    var_tot_words = df_count['Words'].sum()
    #calculate the averages (mean)
    var_av_paras = round(var_tot_paras/var_no_int, 1)
    var_av_words = round(var_tot_words/var_no_int,1)
    var_av_words_per_para = round(var_tot_words/var_tot_paras, 1)
    #calculate the median
    var_med_paras = df_count['Paragraphs'].median()
    var_med_words = df_count['Words'].median()
    
    #append row to dataframe
    df_country_overview = df_country_overview.append({'Country':x, 'No_interviews':var_no_int,
                    'Total_Paragraphs':var_tot_paras, 'Total_Words':var_tot_words, \
                    'Median_Para':var_med_paras, 'Median_Words':var_med_words, 'av_paras_per_int':var_av_paras, 
                    'av_words_per_int':var_av_words, 'av_words_per_para':var_av_words_per_para}, ignore_index=True)
    
    #state the interviews for each actor group
    dict_class_no = {}
    for y in set(df_interviews_overview['Class']):
        dict_class_no[y] = len(df_count[df_count['Class']==y])
    var_last = len(df_country_overview)-1
    for z in dict_class_no:
        df_country_overview.loc[var_last, z] = dict_class_no[z]
    
#include a row with the Total of all countries 
var_TOTAL_countries = len(df_country_overview) #number of countries in the dataframe
var_ind = var_TOTAL_countries-1 #index of the last country

var_TOTAL_int = df_country_overview.loc[:var_ind ,'No_interviews'].sum()
var_TOTAL_paras = df_country_overview.loc[:var_ind, 'Total_Paragraphs'].sum()
var_TOTAL_words = df_country_overview.loc[:var_ind, 'Total_Words'].sum()
var_TOTAL_med_paras = df_country_overview.loc[:var_ind, 'Median_Para'].median()
var_TOTAL_med_words = df_country_overview.loc[:var_ind, 'Median_Words'].median()
    
#append row to dataframe
df_country_overview = df_country_overview.append({'Country':'TOTAL', 'No_interviews':var_TOTAL_int,
        'Total_Paragraphs':var_TOTAL_paras, 'Total_Words':var_TOTAL_words,
        'Median_Para':var_TOTAL_med_paras, 'Median_Words':var_TOTAL_med_words,                                           
        'av_paras_per_int':round(var_TOTAL_paras/var_TOTAL_int, 1),
        'av_words_per_int':round(var_TOTAL_words/var_TOTAL_int, 1),
        'av_words_per_para':round(var_TOTAL_words/var_TOTAL_paras,1)}, ignore_index=True)
for z in dict_class_no:
    df_country_overview.loc[var_TOTAL_countries, z] = df_country_overview.loc[:var_TOTAL_countries, z].sum()

In [6]:
df_country_overview.to_csv('.\\results\overview_interviews\qa_overview_interviews_by_country_'+timestr+'.csv',
        sep=';', decimal=',')

## grouped by actor classification

In [7]:
#calculate per actors group mean paragraph number, min&max, words per paragraph average etc.
df_class_overview = pd.DataFrame()
for x in set(df_interviews_overview['Class']):
    df_class = df_interviews_overview[df_interviews_overview['Class']==x]
    var_no_int = len(df_class)
    var_tot_paras = df_class['Paragraphs'].sum()
    var_tot_words = df_class['Words'].sum()
    var_av_paras = round(var_tot_paras/var_no_int, 1)
    var_av_words = round(var_tot_words/var_no_int,1)
    var_av_words_per_para = round(var_tot_words/var_tot_paras, 1)
    #calculate the median
    var_med_paras = df_class['Paragraphs'].median()
    var_med_words = df_class['Words'].median()
    
    #append row to dataframe
    df_class_overview = df_class_overview.append({'Class':x, 'No_interviews':var_no_int,
                    'Total_Paragraphs':var_tot_paras, 'Total_Words':var_tot_words, 
                    'Median_Para':var_med_paras, 'Median_Words':var_med_words, 'Average_paras_per_int':var_av_paras, 
                    'Average_words_per_int':var_av_words, 'Average_words_per_para':var_av_words_per_para}, ignore_index=True)

    #state the interviews for each country
    dict_count_no = {}
    for y in set(df_interviews_overview['Country']):
        dict_count_no[y] = len(df_class[df_class['Country']==y])
    var_last = len(df_class_overview)-1
    for z in dict_count_no:
        df_class_overview.loc[var_last, z] = dict_count_no[z]
        
#include a row with the Total of all actor classification groups
var_TOTAL_class = len(df_class_overview) #number of countries in the dataframe
var_ind = var_TOTAL_class-1 #index of the last country

var_TOTAL_int = df_class_overview.loc[:var_ind ,'No_interviews'].sum()
var_TOTAL_paras = df_class_overview.loc[:var_ind, 'Total_Paragraphs'].sum()
var_TOTAL_words = df_class_overview.loc[:var_ind, 'Total_Words'].sum()

var_TOTAL_med_paras = df_class_overview.loc[:var_ind, 'Median_Para'].median()
var_TOTAL_med_words = df_class_overview.loc[:var_ind, 'Median_Words'].median()

#append row to dataframe
df_class_overview = df_class_overview.append({'Class':'TOTAL', 'No_interviews':var_TOTAL_int,
        'Total_Paragraphs':var_TOTAL_paras, 'Total_Words':var_TOTAL_words,
        'Median_Para':var_TOTAL_med_paras, 'Median_Words':var_TOTAL_med_words,                                           
        'Average_paras_per_int':round(var_TOTAL_paras/var_TOTAL_int, 1),
        'Average_words_per_int':round(var_TOTAL_words/var_TOTAL_int, 1),
        'Average_words_per_para':round(var_TOTAL_words/var_TOTAL_paras,1)}, ignore_index=True)
for z in dict_count_no:
    df_class_overview.loc[var_TOTAL_class, z] = df_class_overview.loc[:var_TOTAL_class, z].sum()

In [8]:
df_class_overview.to_csv('.\\results\overview_interviews\qa_overview_interviews_by_classification_'+timestr+'.csv',
    sep=';', decimal=',')